---
title: "Approximate Graph Symmetry Methods"
author: "Dávid Kubek"
format:
  html:
    toc: true
    code-fold: true
    code-summary: "Show the code"
    embed-resources: true
    smooth-scroll: true
---

# Continuous Methods for Approximate Graph Symmetries

This document provides a demonstration of the five optimization methods
implemented for the Master's thesis, "Using relaxation to compute approximate
symmetry of graphs."

The notebook is self-contained. It first generates a synthetic graph with a
known, built-in approximate symmetry and then runs each method to find this
symmetry. The results are visualized to allow for a direct comparison of their
performance on this controlled example.


## 1. Setup and Imports

First, we import the necessary libraries for data manipulation and
visualization, as well as the five implemented solution methods from the `AGS`
package.

In [ ]:
#| label: setup
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import random as sparse_random

import random

# Import the five implemented methods
from AGS.methods import (
    QSA,
    InteriorPoint,
    Manifold,
    OrthogonalRelaxation,
    DimensionalityReduction
)

# Set a consistent style for plots
sns.set_theme(style="white", context="notebook")

## 2. Synthetic Data Generation

For the sake of example, we create a synthetic LRM graph with a known, imperfect symmetry. 

In [ ]:
#| label: generate-data

def generate_lrm_graph(
    n: int, p: float, q: float, k: int = 0, seed: int | None = None
) -> nx.Graph:
    """
    Generate a Lateral Random Model (LRM) graph with k-rewiring operations.

    The LRM consists of two coupled Erdős-Rényi graphs with controlled
    bilateral symmetry that can be disrupted through rewiring.

    Parameters
    ----------
    n : int
        Total number of vertices (must be even)
    p : float
        Intra-component edge probability (within each ER component)
    q : float
        Inter-component edge probability (between corresponding vertices)
    k : int
        Number of rewiring operations to perform
    seed : int, optional
        Random seed for reproducibility

    Returns
    -------
    G : nx.Graph
        The generated LRM graph with k rewirings applied

    Raises
    ------
    ValueError
        If n is not even or if probabilities are not in [0,1]
    """
    if n % 2 != 0:
        raise ValueError("Total number of vertices n must be even")
    if not (0 <= p <= 1) or not (0 <= q <= 1):
        raise ValueError("Probabilities p and q must be in [0,1]")

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    n_half = n // 2

    # Step 1: Generate the first ER component G
    G_left = nx.erdos_renyi_graph(n_half, p, seed=seed)

    # Step 2: Create the complete LRM graph
    L = nx.Graph()

    # Add vertices for both components
    # Left component: vertices 0 to n_half-1
    # Right component: vertices n_half to n-1
    L.add_nodes_from(range(n))

    # Add edges from the left component
    for u, v in G_left.edges():
        L.add_edge(u, v)

    # Add edges for the right component (exact copy of left)
    for u, v in G_left.edges():
        L.add_edge(u + n_half, v + n_half)

    # Step 3: Add inter-component edges with probability q
    for i in range(n_half):
        if random.random() < q:
            L.add_edge(i, i + n_half)

    # Step 4: Apply k rewiring operations
    for _ in range(k):
        _rewire_once(L)

    return L


def _rewire_once(G: nx.Graph) -> None:
    """
    Perform a single rewiring operation on the graph.

    Removes a randomly selected edge and adds a new edge between
    two previously unconnected vertices.
    """
    if G.number_of_edges() == 0:
        return

    # Get list of edges and non-edges
    edges = list(G.edges())
    n = G.number_of_nodes()

    # Find all possible non-edges (vertex pairs without an edge)
    non_edges = []
    for u in range(n):
        for v in range(u + 1, n):
            if not G.has_edge(u, v):
                non_edges.append((u, v))

    if len(non_edges) == 0:
        # Graph is complete, cannot add new edges
        return

    # Remove a random edge
    edge_to_remove = random.choice(edges)
    G.remove_edge(*edge_to_remove)

    # Add a random non-edge
    edge_to_add = random.choice(non_edges)
    G.add_edge(*edge_to_add)

In [ ]:
# Generate our graph
A_synthetic = nx.adjacency_matrix(generate_lrm_graph(100, 0.1, 0.1)).toarray()

In [ ]:
# Visualize the adjacency matrix
sns.heatmap(A_synthetic)
plt.title(f"Adjacency Matrix of Synthetic Graph")
plt.show()

The heatmap above shows the structure of our test graph. The two bright squares
on the diagonal represent the two nearly-identical subgraphs. The goal for our
algorithms is to find the permutation that swaps these two halves.

## 3. Running the Optimization Methods

We will now instantiate and run each of the five implemented methods on our
synthetic graph. For each method, we will use a set of default parameters known
to perform reasonably well. 

The output of the methods is the final output of the methods before being
projected to the DS manifold.

### 3.1. Quadratic Symmetry Approximator (QSA)

QSA is the first-order baseline method using the Frank-Wolfe algorithm.

In [ ]:
#| label: run-qsa
qsa_method = QSA(max_iter=500, verbose=0)
results_qsa = qsa_method.solve(A_synthetic, c=0.1)
P_qsa = results_qsa["P"]

print(f"QSA completed in {results_qsa['metrics']['time']:.2f} seconds.")

sns.heatmap(P_qsa)
plt.show()

### 3.2. Interior-Point Method (IP)

IP is a second-order method that uses the IPOPT solver. This method is powerful
but computationally more intensive.

In [ ]:
#| label: run-ip
ip_method = InteriorPoint(max_iter=500, verbose=0)
results_ip = ip_method.solve(A_synthetic, c=0.8)
P_ip = results_ip["P"]

print(f"Interior-Point method completed in {results_ip['metrics']['time']:.2f} seconds.")

sns.heatmap(P_ip)
plt.show()

### 3.3. Manifold Optimization (MF)

MF is a geometry-aware method that performs optimization on the Doubly
Stochastic manifold.

In [ ]:
#| label: run-mf
mf_method = Manifold(optimizer="steepest_descent", max_iter=500)
results_mf = mf_method.solve(A_synthetic, c=0.5)
P_mf = results_mf["P"]

print(f"Manifold Optimization completed in {results_mf['metrics']['time']:.2f} seconds.")

sns.heatmap(P_mf)
plt.show()

### 3.4. Orthogonal Relaxation (OR)

OR uses an alternative relaxation space (the orthogonal group) and is
parameterized by temperature.

In [ ]:
#| label: run-or
or_method = OrthogonalRelaxation(
    max_iter=3000,
    initial_tau=0.7,
    final_tau=None,
    loss="convex",
    verbose=1
)
results_or = or_method.solve(A_synthetic, c=0.1)
P_or = results_or["P"]

print(f"Orthogonal Relaxation completed in {results_or['metrics']['time']:.2f} seconds.")

sns.heatmap(P_or)
plt.show()

### 3.5. Dimensionality Reduction (DR)

DR uses the `SoftSort` operator to reduce the optimization space to $O(n)$.

In [ ]:
#| label: run-dr
dr_method = DimensionalityReduction(
    max_iter=3000,
    initial_tau=0.1,
    final_tau=None, # Stronger relaxation performs better for DR
    loss="convex",
    verbose=1
)
results_dr = dr_method.solve(A_synthetic, c=0.1)
P_dr = results_dr["P"]

print(f"Dimensionality Reduction completed in {results_dr['metrics']['time']:.2f} seconds.")

sns.heatmap(P_dr)
plt.show()

## 4. Comparative Analysis of Results

Now, we visualize the resulting permutation matrices from all five methods
side-by-side. A perfect result would be a permutation swapping the two graph
halves. 

In [ ]:
#| label: comparison-plot

# Collect results
methods = {
    "QSA": P_qsa,
    "Interior-Point": P_ip,
    "Manifold": P_mf,
    "Orthogonal Relaxation": P_or,
    "Dimensionality Reduction": P_dr
}

fig, axes = plt.subplots(1, 5, figsize=(15, 3))
fig.suptitle("Comparison of Resulting Permutation Matrices", fontsize=16, y=1.05)

for i, (name, P) in enumerate(methods.items()):
    # Project the doubly stochastic matrix to a hard permutation for clear visualization
    from scipy.optimize import linear_sum_assignment
    rows, cols = linear_sum_assignment(-P)
    P_hard = np.zeros_like(P)
    P_hard[rows, cols] = 1
    
    sns.heatmap(P_hard, ax=axes[i], cbar=False, cmap="viridis", square=True)
    axes[i].set_title(name)
    axes[i].set_xticks([])
    axes[i].set_yticks([])

plt.tight_layout()
plt.show()